In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

In [33]:
class MF():
    #Initializing the user-movie rating matrix, no. of latent features, alpha and beta
    def __init__(self,R,T, K, alpha,beta,iterations, bias=1):
        self.bias=bias
        self.R=R ## rating train matrix
        self.num_users,self.num_items = R.shape
        self.K = K ## number of latent features
        self.T = T ## traing train matrix
        self.alpha=alpha ## learning rate
        self.beta=beta ## coefficient of regularization
        self.iterations = iterations ## number of iter
        self.b_u = np.zeros(self.num_users) ## bias of users
        self.b_i = np.zeros(self.num_items) ## bias of item
        self.b = np.mean(self.R[np.where(self.R!=0)]) ## initializing bias
     
    def train(self):
         #Initializing user-feature and movie-feature matrix  
        self.P = np.random.normal(scale=1./self.K, size = (self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        #Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R!=0)])
        
        #List of training samples
        self.samples = [
            (i,j,self.R[i,j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i,j] > 0
        ]
        
        #Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            ## SGD for update parameters
            self.sgd()
            rmse_train, rmse_test = self.rmse()
            training_process.append((i, rmse_train, rmse_test))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; rmse_train = %.4f ;rmse_test = %.4f" % (i+1,rmse_train, rmse_test))
        return training_process

    
    #Computing total root mean squared error of train and test
    def rmse(self):
        ## get rating train and test
        xs_test,ys_test = self.T.nonzero()
        xs_train, ys_train = self.R.nonzero()
        
        predicted = self.full_matrix()
        error_test = 0
        error_train = 0
        
        ## count number of ratings train and test
        count_test = np.count_nonzero(T)
        count_train = np.count_nonzero(R)
        
        ## calculate error on test_data
        for x,y in zip(xs_test,ys_test):
            error_test += pow(self.T[x,y] - predicted[x,y],2)
        error_test /= count_test
        
        ## calculate error on train_data
        for x,y in zip(xs_train,ys_train):
            error_train += pow(self.R[x,y] - predicted[x,y],2)
        error_train /= count_train
        return np.sqrt(error_train), np.sqrt(error_test)
        
    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            if self.bias:
                self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
                self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
        
    #Ratings for user i and movie j
    def get_rating(self,i,j):
        if self.bias:
            prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        else:
            prediction = self.P[i, :].dot(self.Q[j, :].T)
        return prediction
                      
    # Full user-movie rating matrix
    def full_matrix(self):
        if self.bias:
            return mf.b + mf.b_u[:, np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)     
        return mf.P.dot(mf.Q.T)
    
    def recommend(self, u, countItem = 0):           
        recommended_items = []
        for i in range(self.num_items):
            if R[u-1][i] > 0:
                continue
            rating = self.get_rating(u-1,i) - self.b_u[u-1] - self.b
            if rating > 0: 
                recommended_items.append((i + 1,rating))
        if countItem > 0:
            return list(zip(*sorted(recommended_items, key=lambda data: data[1], reverse=True)[:countItem]))[0] 
        return recommended_items

In [34]:
from utils import *
all_ratings_train, ratings_train, ratings_valid, ratings_test = split_dataset()
columns = ["user_id", "movie_id", "rating", "unix_timestamp"]
all_ratings_train = pd.DataFrame(all_ratings_train, columns=columns)
ratings_train = pd.DataFrame(ratings_train, columns=columns)
ratings_valid = pd.DataFrame(ratings_valid, columns=columns)
ratings_test = pd.DataFrame(ratings_test, columns=columns)

n_users = int(np.max(ratings_train.iloc[:, 0])) 
n_items = int(np.max(ratings_train.iloc[:, 1]))

# Predict all missing ratings:
R= np.array(ratings_train.pivot(index = 'user_id', columns ='movie_id', values = 'rating').reindex(np.arange(1, n_items+1), axis ='columns').fillna(0))
T= np.array(ratings_valid.pivot(index = 'user_id', columns ='movie_id', values = 'rating').reindex(np.arange(1, n_items+1), axis ='columns').fillna(0))

In [19]:
mf = MF(R,T, K=30, alpha=0.001, beta=0.005, iterations=100, bias=0)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; rmse_train = 1.6288 ;rmse_test = 1.6745
Iteration: 40 ; rmse_train = 1.0261 ;rmse_test = 1.0742
Iteration: 60 ; rmse_train = 0.9411 ;rmse_test = 0.9967
Iteration: 80 ; rmse_train = 0.9016 ;rmse_test = 0.9697
Iteration: 100 ; rmse_train = 0.8676 ;rmse_test = 0.9549

P x Q:
[[ 3.93375388  3.40777472  3.16110236 ... -0.00905951  0.92755891
   0.95868091]
 [ 4.14983605  3.48426515  3.27913243 ... -0.03516614  0.95195688
   0.9269826 ]
 [ 3.18180861  2.74828821  2.57661306 ... -0.02852723  0.74118428
   0.74218979]
 ...
 [ 3.95007115  3.32360102  3.17394767 ... -0.01663898  0.90574369
   0.87525293]
 [ 4.23385635  3.61629673  3.3830447  ... -0.02372774  1.03188148
   0.96907275]
 [ 3.75960937  3.29214824  3.21846595 ... -0.04183099  0.82964705
   0.88413724]]



In [7]:
mf.recommend(1, countItem = 10)

(511, 919, 430, 408, 302, 474, 647, 171, 513, 530)

In [10]:
np.array(training_process)[:, 1].argmin()

73

In [38]:
np.random.seed(42)
for k in range(10, 51, 10):
    mf = MF(R,T, K=k, alpha=0.001, beta=0.005, iterations=100, bias=0)
    print(k)
    training_process = mf.train()

10
Iteration: 20 ; rmse_train = 1.5252 ;rmse_test = 1.5776
Iteration: 40 ; rmse_train = 1.0113 ;rmse_test = 1.0673
Iteration: 60 ; rmse_train = 0.9305 ;rmse_test = 0.9971
Iteration: 80 ; rmse_train = 0.8912 ;rmse_test = 0.9733
Iteration: 100 ; rmse_train = 0.8591 ;rmse_test = 0.9616
20
Iteration: 20 ; rmse_train = 1.5734 ;rmse_test = 1.6199
Iteration: 40 ; rmse_train = 1.0196 ;rmse_test = 1.0697
Iteration: 60 ; rmse_train = 0.9365 ;rmse_test = 0.9956
Iteration: 80 ; rmse_train = 0.8957 ;rmse_test = 0.9691
Iteration: 100 ; rmse_train = 0.8603 ;rmse_test = 0.9549
30
Iteration: 20 ; rmse_train = 1.5825 ;rmse_test = 1.6286
Iteration: 40 ; rmse_train = 1.0236 ;rmse_test = 1.0715
Iteration: 60 ; rmse_train = 0.9423 ;rmse_test = 0.9968
Iteration: 80 ; rmse_train = 0.9038 ;rmse_test = 0.9705
Iteration: 100 ; rmse_train = 0.8684 ;rmse_test = 0.9547
40
Iteration: 20 ; rmse_train = 1.6244 ;rmse_test = 1.6690
Iteration: 40 ; rmse_train = 1.0292 ;rmse_test = 1.0754
Iteration: 60 ; rmse_train = 0.94

In [39]:
np.random.seed(42)
for k in range(10, 51, 10):
    mf = MF(R,T, K=k, alpha=0.001, beta=0.005, iterations=100, bias=1)
    print(k)
    training_process = mf.train()

10
Iteration: 20 ; rmse_train = 0.9430 ;rmse_test = 0.9668
Iteration: 40 ; rmse_train = 0.9199 ;rmse_test = 0.9550
Iteration: 60 ; rmse_train = 0.9060 ;rmse_test = 0.9510
Iteration: 80 ; rmse_train = 0.8917 ;rmse_test = 0.9493
Iteration: 100 ; rmse_train = 0.8730 ;rmse_test = 0.9488
20
Iteration: 20 ; rmse_train = 0.9444 ;rmse_test = 0.9658
Iteration: 40 ; rmse_train = 0.9231 ;rmse_test = 0.9539
Iteration: 60 ; rmse_train = 0.9114 ;rmse_test = 0.9494
Iteration: 80 ; rmse_train = 0.8996 ;rmse_test = 0.9467
Iteration: 100 ; rmse_train = 0.8824 ;rmse_test = 0.9438
30
Iteration: 20 ; rmse_train = 0.9450 ;rmse_test = 0.9657
Iteration: 40 ; rmse_train = 0.9246 ;rmse_test = 0.9538
Iteration: 60 ; rmse_train = 0.9141 ;rmse_test = 0.9494
Iteration: 80 ; rmse_train = 0.9045 ;rmse_test = 0.9469
Iteration: 100 ; rmse_train = 0.8908 ;rmse_test = 0.9443
40
Iteration: 20 ; rmse_train = 0.9453 ;rmse_test = 0.9657
Iteration: 40 ; rmse_train = 0.9253 ;rmse_test = 0.9539
Iteration: 60 ; rmse_train = 0.91

In [40]:
np.random.seed(42)
alphas = [0.002, 0.003,  0.005, 0.01]
for a in alphas:
    mf = MF(R,T, K=20, alpha=a, beta=0.005, iterations=100, bias=0)
    print(a)
    training_process = mf.train()

0.002
Iteration: 20 ; rmse_train = 1.0186 ;rmse_test = 1.0681
Iteration: 40 ; rmse_train = 0.9000 ;rmse_test = 0.9710
Iteration: 60 ; rmse_train = 0.8281 ;rmse_test = 0.9489
Iteration: 80 ; rmse_train = 0.7559 ;rmse_test = 0.9483
Iteration: 100 ; rmse_train = 0.6943 ;rmse_test = 0.9630
0.003
Iteration: 20 ; rmse_train = 0.9371 ;rmse_test = 0.9964
Iteration: 40 ; rmse_train = 0.8237 ;rmse_test = 0.9479
Iteration: 60 ; rmse_train = 0.7199 ;rmse_test = 0.9530
Iteration: 80 ; rmse_train = 0.6449 ;rmse_test = 0.9787
Iteration: 100 ; rmse_train = 0.5956 ;rmse_test = 1.0053
0.005
Iteration: 20 ; rmse_train = 0.8626 ;rmse_test = 0.9580
Iteration: 40 ; rmse_train = 0.6898 ;rmse_test = 0.9665
Iteration: 60 ; rmse_train = 0.5938 ;rmse_test = 1.0180
Iteration: 80 ; rmse_train = 0.5443 ;rmse_test = 1.0580
Iteration: 100 ; rmse_train = 0.5152 ;rmse_test = 1.0867
0.01
Iteration: 20 ; rmse_train = 0.6907 ;rmse_test = 0.9716
Iteration: 40 ; rmse_train = 0.5476 ;rmse_test = 1.0576
Iteration: 60 ; rmse_t

In [41]:
np.random.seed(42)
alphas = [0.002, 0.003,  0.005, 0.01]
for a in alphas:
    mf = MF(R,T, K=20, alpha=a, beta=0.005, iterations=100, bias=1)
    print(a)
    training_process = mf.train()

0.002
Iteration: 20 ; rmse_train = 0.9231 ;rmse_test = 0.9538
Iteration: 40 ; rmse_train = 0.8997 ;rmse_test = 0.9468
Iteration: 60 ; rmse_train = 0.8563 ;rmse_test = 0.9409
Iteration: 80 ; rmse_train = 0.7862 ;rmse_test = 0.9391
Iteration: 100 ; rmse_train = 0.7181 ;rmse_test = 0.9519
0.003
Iteration: 20 ; rmse_train = 0.9112 ;rmse_test = 0.9495
Iteration: 40 ; rmse_train = 0.8558 ;rmse_test = 0.9407
Iteration: 60 ; rmse_train = 0.7525 ;rmse_test = 0.9440
Iteration: 80 ; rmse_train = 0.6633 ;rmse_test = 0.9717
Iteration: 100 ; rmse_train = 0.6043 ;rmse_test = 1.0025
0.005
Iteration: 20 ; rmse_train = 0.8830 ;rmse_test = 0.9457
Iteration: 40 ; rmse_train = 0.7147 ;rmse_test = 0.9579
Iteration: 60 ; rmse_train = 0.5997 ;rmse_test = 1.0141
Iteration: 80 ; rmse_train = 0.5412 ;rmse_test = 1.0588
Iteration: 100 ; rmse_train = 0.5073 ;rmse_test = 1.0917
0.01
Iteration: 20 ; rmse_train = 0.7144 ;rmse_test = 0.9535
Iteration: 40 ; rmse_train = 0.5411 ;rmse_test = 1.0534
Iteration: 60 ; rmse_t

In [42]:
np.random.seed(42)
betas = [0.001, 0.002, 0.01, 0.02]
for b in betas:
    mf = MF(R,T, K=20, alpha=0.001, beta=b, iterations=100, bias=0)
    print(b)
    training_process = mf.train()

0.001
Iteration: 20 ; rmse_train = 1.5383 ;rmse_test = 1.5857
Iteration: 40 ; rmse_train = 1.0182 ;rmse_test = 1.0681
Iteration: 60 ; rmse_train = 0.9380 ;rmse_test = 0.9964
Iteration: 80 ; rmse_train = 0.8976 ;rmse_test = 0.9707
Iteration: 100 ; rmse_train = 0.8605 ;rmse_test = 0.9564
0.002
Iteration: 20 ; rmse_train = 1.5935 ;rmse_test = 1.6397
Iteration: 40 ; rmse_train = 1.0202 ;rmse_test = 1.0706
Iteration: 60 ; rmse_train = 0.9360 ;rmse_test = 0.9960
Iteration: 80 ; rmse_train = 0.8941 ;rmse_test = 0.9695
Iteration: 100 ; rmse_train = 0.8569 ;rmse_test = 0.9553
0.01
Iteration: 20 ; rmse_train = 1.5537 ;rmse_test = 1.6013
Iteration: 40 ; rmse_train = 1.0187 ;rmse_test = 1.0689
Iteration: 60 ; rmse_train = 0.9383 ;rmse_test = 0.9964
Iteration: 80 ; rmse_train = 0.9000 ;rmse_test = 0.9709
Iteration: 100 ; rmse_train = 0.8664 ;rmse_test = 0.9564
0.02
Iteration: 20 ; rmse_train = 1.5818 ;rmse_test = 1.6282
Iteration: 40 ; rmse_train = 1.0224 ;rmse_test = 1.0719
Iteration: 60 ; rmse_tr

In [46]:
np.random.seed(42)
betas = [0.001, 0.002, 0.01, 0.02]
for a in betas:
    mf = MF(R,T, K=20, alpha=0.001, beta=a, iterations=100, bias=1)
    print(a)
    training_process = mf.train()

0.001
Iteration: 20 ; rmse_train = 0.9442 ;rmse_test = 0.9656
Iteration: 40 ; rmse_train = 0.9228 ;rmse_test = 0.9537
Iteration: 60 ; rmse_train = 0.9108 ;rmse_test = 0.9494
Iteration: 80 ; rmse_train = 0.8982 ;rmse_test = 0.9469
Iteration: 100 ; rmse_train = 0.8795 ;rmse_test = 0.9442
0.002
Iteration: 20 ; rmse_train = 0.9442 ;rmse_test = 0.9658
Iteration: 40 ; rmse_train = 0.9228 ;rmse_test = 0.9539
Iteration: 60 ; rmse_train = 0.9108 ;rmse_test = 0.9495
Iteration: 80 ; rmse_train = 0.8982 ;rmse_test = 0.9469
Iteration: 100 ; rmse_train = 0.8796 ;rmse_test = 0.9441
0.01
Iteration: 20 ; rmse_train = 0.9445 ;rmse_test = 0.9661
Iteration: 40 ; rmse_train = 0.9235 ;rmse_test = 0.9542
Iteration: 60 ; rmse_train = 0.9122 ;rmse_test = 0.9499
Iteration: 80 ; rmse_train = 0.9015 ;rmse_test = 0.9476
Iteration: 100 ; rmse_train = 0.8866 ;rmse_test = 0.9456
0.02
Iteration: 20 ; rmse_train = 0.9449 ;rmse_test = 0.9660
Iteration: 40 ; rmse_train = 0.9241 ;rmse_test = 0.9539
Iteration: 60 ; rmse_tr

In [47]:
np.random.seed(42)
mf = MF(R,T, K=20, alpha=0.001, beta=0.005, iterations=200, bias=1)
training_process = mf.train()

Iteration: 20 ; rmse_train = 0.9444 ;rmse_test = 0.9656
Iteration: 40 ; rmse_train = 0.9231 ;rmse_test = 0.9537
Iteration: 60 ; rmse_train = 0.9114 ;rmse_test = 0.9494
Iteration: 80 ; rmse_train = 0.8997 ;rmse_test = 0.9469
Iteration: 100 ; rmse_train = 0.8826 ;rmse_test = 0.9442
Iteration: 120 ; rmse_train = 0.8564 ;rmse_test = 0.9409
Iteration: 140 ; rmse_train = 0.8225 ;rmse_test = 0.9383
Iteration: 160 ; rmse_train = 0.7865 ;rmse_test = 0.9390
Iteration: 180 ; rmse_train = 0.7513 ;rmse_test = 0.9437
Iteration: 200 ; rmse_train = 0.7185 ;rmse_test = 0.9519


In [48]:
np.array(training_process)[:, 2].argmin()

146

In [49]:
training_process[150]

(150, 0.8027273496426064, 0.9382031684385453)

In [50]:
np.random.seed(42)
mf = MF(R,T, K=20, alpha=0.001, beta=0.005, iterations=200, bias=0)
training_process = mf.train()

Iteration: 20 ; rmse_train = 1.5404 ;rmse_test = 1.5877
Iteration: 40 ; rmse_train = 1.0189 ;rmse_test = 1.0686
Iteration: 60 ; rmse_train = 0.9391 ;rmse_test = 0.9968
Iteration: 80 ; rmse_train = 0.8999 ;rmse_test = 0.9711
Iteration: 100 ; rmse_train = 0.8645 ;rmse_test = 0.9565
Iteration: 120 ; rmse_train = 0.8282 ;rmse_test = 0.9485
Iteration: 140 ; rmse_train = 0.7916 ;rmse_test = 0.9460
Iteration: 160 ; rmse_train = 0.7563 ;rmse_test = 0.9479
Iteration: 180 ; rmse_train = 0.7238 ;rmse_test = 0.9540
Iteration: 200 ; rmse_train = 0.6948 ;rmse_test = 0.9627


In [52]:
training_process[150]

(150, 0.771892491596147, 0.9464578087055523)

In [53]:
R= np.array(all_ratings_train.pivot(index = 'user_id', columns ='movie_id', values = 'rating').reindex(np.arange(1, n_items+1), axis ='columns').fillna(0))
T= np.array(ratings_test.pivot(index = 'user_id', columns ='movie_id', values = 'rating').reindex(np.arange(1, n_items+1), axis ='columns').fillna(0))
np.random.seed(42)
mf = MF(R,T, K=20, alpha=0.001, beta=0.005, iterations=150, bias=1)
training_process = mf.train()
training_process[-1]

Iteration: 20 ; rmse_train = 0.9403 ;rmse_test = 0.9578
Iteration: 40 ; rmse_train = 0.9218 ;rmse_test = 0.9455
Iteration: 60 ; rmse_train = 0.9107 ;rmse_test = 0.9408
Iteration: 80 ; rmse_train = 0.8967 ;rmse_test = 0.9367
Iteration: 100 ; rmse_train = 0.8729 ;rmse_test = 0.9306
Iteration: 120 ; rmse_train = 0.8391 ;rmse_test = 0.9244
Iteration: 140 ; rmse_train = 0.8026 ;rmse_test = 0.9229


(149, 0.7846477828794628, 0.9240660518641352)